In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
#df.drop('Name', axis=1, inplace=True)
#df.drop('PassengerId', axis=1, inplace=True)

def process_dataframe(df): # function to apply identified changes to the dataframe

    target = 'Transported'

    cabins = df['Cabin']
    cabin_block = []
    cabin_num = []
    cabin_side = []
    for room in cabins:
        if type(room) != float:
            cabin_block.append(room[0])

            cabin_num.append(room[2:-2])

            cabin_side.append(room[-1])
        else:
            cabin_block.append('-')
            cabin_num.append(-1)# specifically for cabin number , converted to - below
            cabin_side.append('-')

    df['cabin_block'] = cabin_block
    df['cabin_num'] = cabin_num
    df['cabin_side'] = cabin_side


    column = 'cabin_num'
    num_sections =[]
    cabin_nums = df[column]

    for amount in cabin_nums:
        amount = int(amount)
        if amount < 0:
            num_sections.append('-')
        elif amount < 275:
            num_sections.append('0-275')
        elif amount < 625:
            num_sections.append('275-625')
        elif amount < 1175:
            num_sections.append('625-1125')
        else:
            num_sections.append('1125-')

    df['cabin_num_section'] = num_sections






    VIPs = df['VIP']

    new_VIPs = [(val ==True) for val in VIPs]
    df['VIP'] = new_VIPs



    spending_columns = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']

    for column in spending_columns:
        new_column = []

        values = df[column]

        for value in values:

            if str(value) == 'nan':
                new_column.append(0)
                continue
            new_column.append(int(value**(1/7)))

        df[column]= new_column





    categorical_columns =['cabin_block','cabin_num_section','cabin_side','HomePlanet','CryoSleep','Destination','VIP']

    age = 'Age'


    for column in categorical_columns:
        column_data = df[column]

        new_column_data = []
        for data in column_data:
            if str(data) == 'nan':
                new_column_data.append('-')
                continue
            new_column_data.append(data)

        df[column] = new_column_data


    new_ages = [25 if str(data) == 'nan' else data for data in df['Age']]
    df['Age'] = new_ages



    one_hot_columns = []
    for column in categorical_columns:

        values = df[column]

        unique_values = list(df[column].unique())
        if '-' in unique_values:
            unique_values.remove('-')

        for current_value in unique_values:
            one_hot = []

            for value in values:
                if value == current_value:
                    one_hot.append(1)
                    continue
                one_hot.append(0)
            df[f'{column}_{current_value}'] =one_hot
            one_hot_columns.append(f'{column}_{current_value}')
            
            
    used_columns = one_hot_columns + spending_columns
    used_columns.append(age)
    used_columns
    return df,used_columns,target





In [3]:
df = pd.read_csv("train.csv")
df.info()
df,used_columns,target = process_dataframe(df)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 42 columns):
 #   Column                      No

In [4]:
y = df[target]
df = df.drop(columns = target)
X = df[used_columns]



In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [6]:
df_test = pd.read_csv("test.csv")
df_test,_,target = process_dataframe(df_test)# we want to give used_columns in the same order as training data, therrefore, used_columns returned here is ignored

X_Final = df_test[used_columns]
#y_test = df_test[target]

In [7]:
df_test.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,...,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,CryoSleep_True,CryoSleep_False,Destination_TRAPPIST-1e,Destination_55 Cancri e,Destination_PSO J318.5-22,VIP_False,VIP_True
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0,0,0,...,1,0,0,1,0,1,0,0,1,0
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0,1,0,...,1,0,0,0,1,1,0,0,1,0
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0,0,0,...,0,1,0,1,0,0,1,0,1,0
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0,3,0,...,0,1,0,0,1,1,0,0,1,0
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,1,0,2,...,1,0,0,0,1,1,0,0,1,0


In [8]:
df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,...,HomePlanet_Europa,HomePlanet_Earth,HomePlanet_Mars,CryoSleep_False,CryoSleep_True,Destination_TRAPPIST-1e,Destination_PSO J318.5-22,Destination_55 Cancri e,VIP_False,VIP_True
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0,0,0,...,1,0,0,1,0,1,0,0,1,0
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,1,1,1,...,0,1,0,1,0,1,0,0,1,0
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,1,3,0,...,1,0,0,1,0,1,0,0,0,1
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0,2,2,...,1,0,0,1,0,1,0,0,1,0
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,2,1,2,...,0,1,0,1,0,1,0,0,1,0


In [9]:
import xgboost as xgb


model_xgb = xgb.XGBClassifier()

model_xgb.fit(X_train,y_train)

from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score

pred = model_xgb.predict(X_train)

print('Performance of Model on training data :')

print('Accuracy: ',accuracy_score(y_train, pred))
print('Precision: ',precision_score(y_train, pred))
print('Recall: ',recall_score(y_train, pred))
print('F1-Score: ',f1_score(y_train, pred))


Performance of Model on training data :
Accuracy:  0.8938373048479868
Precision:  0.895405669599218
Recall:  0.8942401561991539
F1-Score:  0.8948225333767502


In [10]:
pred = model_xgb.predict(X_test)


print('Performance of Model on training data :')

print('Accuracy: ',accuracy_score(y_test, pred))
print('Precision: ',precision_score(y_test, pred))
print('Recall: ',recall_score(y_test, pred))
print('F1-Score: ',f1_score(y_test, pred))

Performance of Model on training data :
Accuracy:  0.8071319018404908
Precision:  0.800599700149925
Recall:  0.8183908045977012
F1-Score:  0.8093974990526714


In [11]:
model_xgb_complete = xgb.XGBClassifier()

model_xgb_complete.fit(X,y)

pred = model_xgb_complete.predict(X_Final)



pred = [bool(p) for p in pred]
df_test[target] = pred
submission_columns = ['PassengerId',target]

df_test[submission_columns].to_csv('submission_tanmaya.csv',index=False)